# Oasis Reinsurance Test Tool v0.2

This notebook allows example reinsurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included
* Basic direct insurance modelling (not the complete engine included in Flamingo)
    * Buildings, Other buildings contents and BI
    * Blanket limit
    * Blanket deductible
    * Site limit
    * Site deductible 
* FAC treaties
* Quota share
* Surplus share
* Per-risk
* Cat XL treaties

## Not included
* Attachement basis
* Aggregate terms
* Multiple portfolios in a single set of input files

## Test cases

A selection of test cases can be found in the examples folder.

* simple_loc_FAC: Single location level fac.
* simple_pol_FAC: Single policy level fac.
* simple_acc_FAC: Single account level fac.
* multiple_FAC: Multiple facs at sameinuring level.
* simple_QS: Single quota share with no risk limits.
* loc_SS: Single surplus share at location level.
* pol_SS: Single surplus share at location level.
* acc_SS: Single surplus share at location level.
* multiple_SS: Multiple surplus shares at same inuring level.
* loc_limit_QS: Single quota share with location level risk limits.
* pol_limit_QS: Single quota share with policy level risk limits.
* acc_limit_QS: Single quota share with account level risk limits.
* multiple_QS_1:  Two quota shares at same inuring level.
* multiple_QS_2:  Two quota shares at different inuring levels.
* simple_CAT_XL: Single cat XL.
* multiple_CAT_XL: Two cat XLs at different inuring levels.


In [ ]:
# Step 1 - general Python code set up
%config IPCompleter.greedy=True
import reinsurance_tester
import jupyter_helper
import pandas as pd

In [ ]:
# Step 2 - load the OED for a worked example, in this case FM Test Case 23.
# Note that only the currently used fields are shown unless show_all is set to True. 
(account_df, location_df, ri_info_df, ri_scope_df, do_reinsurance) = reinsurance_tester.load_oed_dfs('examples/simple_QS', show_all=False)

In [ ]:
# Step 3 - view/edit the account data. 
account_grid = jupyter_helper.show_df(account_df)
account_grid

In [ ]:
# Step 4 - view/edit the location data.
location_grid = jupyter_helper.show_df(location_df)
location_grid

In [ ]:
# Step 5 - view/edit the reinsurance info data.
ri_info_grid = jupyter_helper.show_df(ri_info_df)
ri_info_grid

In [ ]:
# Step 6 - view/edit the reinsurance scope data.
ri_scope_grid = jupyter_helper.show_df(ri_scope_df)
ri_scope_grid

In [ ]:
# Step 7 - pick up any edits in the grid before running the analysis
account_df = account_grid.get_changed_df()
location_df = location_grid.get_changed_df()
ri_info_df = ri_info_grid.get_changed_df()
ri_scope_df = ri_scope_grid.get_changed_df()

In [ ]:
# Step 8 - run the OED data though the Oasis Financial Module and output the losses by item at each inuring level.
net_losses = reinsurance_tester.run_test('test', account_df, location_df, ri_info_df, ri_scope_df, loss_factor=1.0, do_reinsurance=do_reinsurance)
print("Ran {} inuring layers".format(len(net_losses) - 1))
print("Losses for:")
for key in net_losses.keys():
    print("\t{}".format(key))

In [ ]:
# Step 9 - view the direct losses.
key = 'Direct'
net_losses[key]

In [ ]:
# Step 10 - view the losses for the first inuring layer.
key = 'Inuring priority:1 - Risk level:SEL'
net_losses[key]